In [15]:
import re

## this notebook does final checks for a variety of features that we want in Perseus/OGL files but for which Hooktest does not check

legalresps = [
    '<resp>Prepared under the supervision of</resp>',
    '<resp>Digital conversion and editing</resp>',
    '<resp>Prepared under the supervision of</resp>'
]


def testfile(curtg,curwork,curedition):
    fname = 'data/' + curtg + '/' + curwork + '/' + curtg + '.' + curwork + '.' + curedition + '.xml'
    curvolume = ''
    f = open(fname)
    curname = curtg + '.' + curwork
    
    inbody = 0
    for l in f:
        l = re.sub('\s+$','',l)
        
        if(re.search('<body>',l)):
            inbody = 1
            
        m = re.search('<biblScope unit="volume">([0-9]+)',l)
        if(m):
            curvolume = m[1]
            
        m = re.search('(<ref target="http[^"]+)',l)
        if(m and not inbody and not re.search('<ref target="http[s]*://creative',l)):
            booklink = m[1]
            if(re.search('archive.org',booklink)):
                if( not re.search('archive.org[^"]+page/[0-9]+',booklink)):
                    print(curname,'nopagelink',booklink)
               # else:
               #     print(curname,'pagelink',booklink)
            if(re.search('hdl.handle',booklink)):
                if( not re.search('seq=',booklink)):
                    print(curname,'nopagelink',booklink)
                #else:
                #    print(curname,'pagelink',booklink)
            
        m = re.search('(<note[^>]*>)',l)
        if(m):
            curnote = m[1]
            if(not re.search('xml:lang',curnote) and not re.search('<note[^>]*/>',l)):
                print(curtg+'.'+curwork,'note without xml:lang',curnote)
                
        if(re.search('_grc',curedition) and re.search('(<editor role="translator">|type="translation")',l)):
            print(curname,'this is an edition',l)
    
        if(re.search('_(eng|fra|deu)',curedition) and re.search('(<editor>|type="edition")',l)):
            print(curname,'this is a translation',l)
            
        m = re.search('\s*(<resp>.</resp>)',l)
        if(m and not m[1] in legalresps):
            print('<resp>',l)
            
        #make sure we have an n attribute in   <cRefPattern/>
        m = re.search('(<cRefPattern[^>]*>)',l)
        if(m):
            curpat = m[1]
            if(not re.search(' n=',curpat)):
                print(curname,'cRefPattern needs an n attribute')

            
        m = re.search('(<div[^>]+>)',l)
        if(m):
            curdiv = m[1]
            if(re.search('type="textpart"',curdiv) and not re.search('xml:base',curdiv)):
                print(curname,'missing xml:base',curdiv)
                
        m = re.search('(<ref n="[^"]+"/>)',l)
        if(m):
            print(curname,m[1],'merge notes!')
            
        m = re.search('<pb n="([0-9\.]+)"',l)
        if(curvolume and m):
            print(curwork,'vol-page v.N.p.N',l)
            
            
    
    f.close()
    
    
curgroup = 'tlg0062'
curedition = 'perseus-grc3'

for i in range(53,69):
    if(i<10):
        curwork = 'tlg00' + str(i)
    elif(i<100):
        curwork = 'tlg0' + str(i)
    else:
        curwork = 'tlg' + str(i)
    testfile(curgroup,curwork,curedition)
    